In [1]:
%env LD_LIBRARY_PATH=/usr/local/cuda/lib
%env PATH=/usr/local/cuda/bin:/usr/bin

env: LD_LIBRARY_PATH=/usr/local/cuda/lib
env: PATH=/usr/local/cuda/bin:/usr/bin


In [2]:
import tvm
import tvm.testing
from tvm import te
import numpy as np
import random


In [3]:
# # This block is hyperblock settings
M = 1024
K = 2048
N = 1024
# M = te.var("M")
# K = te.var("K")
# N = te.var("N")
'''should schedule matrices of different shapes, so M, K, N are const but not var'''

# dtype = 'float32'

N_dim = 128
MAX_BLOCK_NUM = 32

N_pruned = 768
K_pruned = 384


# block_num = (N_pruned + N_dim - 1) // N_dim
block_num = N_pruned // N_dim
print(type(block_num))

<class 'int'>


In [4]:
# compute K_cal
# K_cal = [K_pruned for _ in range(block_num)]
K_cal = K_pruned

# N_ori = [N // block_num for _ in range(block_num-1)]
# N_ori.append(N - sum(N_ori))
N_ori = N //block_num

# N_cal = [N_dim for _ in range(block_num - 1)]
# N_cal.append(N_pruned - sum(N_cal))
N_cal = N_dim

print(type(K_cal))
print(type(N_ori))
print(type(N_cal))

<class 'int'>
<class 'int'>
<class 'int'>


In [18]:
# mask_k_list = [te.placeholder((K_cal,), name="mask_k_%d"%(i)) for i in range(MAX_BLOCK_NUM)]
# mask_n_list = [te.placeholder((N_cal,), name="mask_n_%d"%(i)) for i in range(MAX_BLOCK_NUM)]
mask_k_list = [te.placeholder((K_cal,), name="mask_k_%d"%(i)) for i in range(block_num)]
mask_n_list = [te.placeholder((N_cal,), name="mask_n_%d"%(i)) for i in range(block_num)]

In [22]:
# input
A_float = te.placeholder((M, K), name='A_float')
A_float_transpose = te.compute((K, M), lambda i,j: A_float[j, i], name='transpose')
'''TODO: write the computing rule of A_float_skip '''
A_float_skip = te.compute((K_cal, M), lambda i, j: A_float_transpose[j, i], name='A_float_skip')
 
# weight
B_float = te.placeholder((K, N), name='B_float')
B_stream_list = []
for bn in range(block_num):
    mask_k = mask_k_list[bn] # `mask_k`'s type is <class 'tvm.te.tensor.Tensor'>
    '''TODO: rewrite the lambda below to compute tiles of B'''
    B_stream_list.append(te.compute((N_cal, K_cal), lambda i, j: B_float[i, j], name="B_stream_%d"%(bn)))

C_float = te.placeholder((M, N), name='C_float')
'''TODO: think of a way to store C_tile'''

In [5]:
# generate random mask
def mask_gen(N: int, N_pruned: int, base = 0):
    mask_keep = list(range(N))
    random.shuffle(mask_keep) # shuffle is an in-place operation
    mask_keep = mask_keep[ : N_pruned]
    mask_keep.sort()
    for i, _ in enumerate(mask_keep):
        mask_keep[i] = mask_keep[i] - i + base
    return mask_keep


[1, 1, 3, 4, 5]


In [9]:
# generate random mask for each block
mask_k_list = []
mask_n_list = []
for i in range(block_num):
    mask_k_list.append(mask_gen(K, K_cal[i]))
    mask_n_list.append(mask_gen(N_ori[i], N_cal[i]))

In [10]:
# compute `off_C` which is the variable name in original cuda code
off_C = [0]
for i in range(1, block_num):
    off_C.append(sum(N_ori[:i]))
print(off_C)

[0, 170, 340, 510, 680, 850]


In [11]:
# def gen_masked_stream_weight(src_tensor: tvm.nd.NDArray, mask_k_list: list, mask_n_list: list , pruned_K, pruned_N, offset):
#     [K, N] = src_tensor.shape
#     dst_tensor = tvm.nd.array(np.zeros((N, K)))
#     for i in range(pruned_K):
#         for j in range(pruned_N):
#             idx_col = mask_k_list[i]+i
#             idx_row = mask_n_list[j]+j+offset
#             dst_tensor[j+offset, i] = src_tensor[idx_col, idx_row]
#     return dst_tensor
'''
`src_tensor` is the weight tensor of the dense model
`dst_tensor` is the input weight of the GPU kernel, 
    which comes from packing and transposing src_tensor
`dst_tensor` corresponds to the `tvm.te.placeholder` defined below 
    and will be feeded to the kernel generated by TVM'''
src_tensor = np.random.rand(K, N) # input tensor of this ipython scrpit
dst_tensor = np.zeros((N, K)) # input tensor of the GPU kernel (batched)
for block_idx in range(block_num):
    offset = sum(N_ori[:block_idx])
    for i in range(K_cal[block_idx]):
        for j in range(N_cal[block_idx]):
            idx_col = mask_k_list[block_idx][i]+i
            idx_row = mask_n_list[block_idx][j]+j+offset
            dst_tensor[j+offset, i] = src_tensor[idx_col, idx_row]
dst_tensor = tvm.nd.array(dst_tensor) # convert a numpy tensor to tvm.nd.array



In [12]:
# calculate the number of zeros and non-zeros in `dst_tensor`
num_zero = 0
num_else = 0
for i in range(N):
    for j in range(K):
        if dst_tensor[i, j]==0:
            num_zero+=1
        else:
            num_else+=1
print(num_zero)
print(num_else)
        

1802240
294912


In [13]:
target = tvm.target.Target(target="cuda", host="llvm")
dev = tvm.device(target.kind.name, 0)

cuda(0)


In [16]:
tgt_gpu = tvm.target.Target(target='cuda', host='llvm')

n = te.var("n")
A = te.placeholder((n, ), name='A')
B = te.placeholder((n, ), name="B")
C = te.compute(A.shape, lambda i : A[i] + B[i], name="C")
print(type(C))

s = te.create_schedule(C.op)

bx, tx = s[C].split(C.op.axis[0], factor=64)

s[C].bind(bx, te.thread_axis("blockIdx.x"))
s[C].bind(tx, te.thread_axis("threadIdx.x"))

fadd = tvm.build(s, [A, B, C], target=tgt_gpu, name='myadd') # [A, B, C]的顺序表示最后生成的kernel中参数传入的顺序
dev = tvm.device(tgt_gpu.kind.name, 0)

n=1024
a = tvm.nd.array(np.random.uniform(size=n).astype(A.dtype), dev)
b = tvm.nd.array(np.random.uniform(size=n).astype(B.dtype), dev)
c = tvm.nd.array(np.zeros(n, dtype=C.dtype), dev)
fadd(a, b, c)
tvm.testing.assert_allclose(c.numpy(), a.numpy()+b.numpy())

if(tgt_gpu.kind.name=='cuda'
    or tgt_gpu.kind.name=='rocm'
    or tgt_gpu.kind.name.startwith('opencl')):
    dev_module = fadd.imported_modules[0]
    print("-----GPU code-----")
    print(fadd.get_source())

    

<class 'tvm.te.tensor.Tensor'>
-----GPU code-----
; ModuleID = 'TVMMod'
source_filename = "TVMMod"
target datalayout = "e-m:e-p270:32:32-p271:32:32-p272:64:64-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-unknown-linux-gnu"

%0 = type { double }
%1 = type { ptr, %2, i32, %3, ptr, ptr, i64 }
%2 = type { i32, i32 }
%3 = type { i8, i8, i16 }

@__tvm_module_ctx = linkonce dllexport local_unnamed_addr global ptr null, align 8
@__TVMFuncCall = linkonce dllexport local_unnamed_addr global ptr null, align 8
@__TVMBackendGetFuncFromEnv = linkonce dllexport local_unnamed_addr global ptr null, align 8
@__TVMAPISetLastError = linkonce dllexport local_unnamed_addr global ptr null, align 8
@.str = private constant [56 x i8] c"Assert fail: num_args == 3, myadd: num_args should be 3\00", align 1
@.str.1 = private constant [123 x i8] c"Assert fail: arg_A_code == 3 or arg_A_code == 13 or arg_A_code == 7 or arg_A_code == 4, myadd: Expect arg[0] to be pointer\00", align 1
@.str.2 = private cons

## deleteRow kernel for A

In [ ]:
'''Input
* source: `A_transposed`, which is defined above
* destination A: [(K_cal, 1)]
* mask: [(K_cal, M)]
* rows'''